In [ ]:
pip install azure-storage-file-datalake pyarrow

In [ ]:
from azure.identity import ManagedIdentityCredential
from azure.storage.filedatalake import DataLakeFileClient
from azure.core.credentials import AccessToken
import time
import pyarrow.parquet as pq
import pyarrow as pa
import tempfile


# Set up authentication
class spoof_token:
    def get_token(*args, **kwargs):
        return AccessToken(
            token=mssparkutils.credentials.getToken(audience="storage"),
            expires_on=int(time.time())+60*10
        )

credential = ManagedIdentityCredential()
credential._credential = spoof_token()

# Set up file client
account_name = "phdidevphi9d194c64"
file_system_name = "source-data"
file_path = "synthetic_patient_mci_seed_data.parquet"
mci_incoming_file_path = f"abfss://{file_system_name}@{account_name}.dfs.core.windows.net/{file_path}"
mci_delta_table_path = f"abfss://{file_system_name}@{account_name}.dfs.core.windows.net/delta/mci"
# file_client = DataLakeFileClient(account_url=f"https://{account_name}.dfs.core.windows.net",
#                                  file_system_name=file_system_name,
#                                  file_path=file_path,
#                                  credential=credential)


def insert(mci_incoming_file_path,mci_delta_table_path):
    df = spark.read.parquet(mci_incoming_file_path)
    df.write.mode("append").format("parquet").save(mci_delta_table_path)
    spark.read.parquet(mci_delta_table_path).show(5)

insert(mci_incoming_file_path,mci_delta_table_path)
        
# # This function inserts incoming MCI data into the MCI delta table
# def insert(bytes):
#     buffer = pa.BufferReader(bytes)
#     parquet_file = pq.ParquetFile(buffer)
#     with tempfile.TemporaryDirectory() as mci_delta_table: 
#         for row in parquet_file.iter_batches(batch_size=1):
#             data = row.to_pylist()[0]
#             # Append another DataFrame into the Parquet file
#             spark.createDataFrame(
#                 [{"person_id": data["person_id"], "incident_id": data["incident_id"],"specimen_collection_date": data["specimen_collection_date"]}]
#             ).write.mode("append").format("parquet").save(mci_delta_table)
                
#         spark.read.parquet(mci_delta_table).show()

# # # # Download parquet file and convert
# download = file_client.download_file()
# bytes = download.readall()

# insert(bytes)


